In [5]:
import time

#opening, reading and closing the file
fd = open("Inventory.txt", "r")
txt = fd.read()
fd.close()

#store products in products variable
products = txt.split('\n')
print(products)
print('\n\n')

updated_products = []

#user_info
ui_name = input("Enter user name       : ")
ui_phone = input("Enter Phone number   : ")
ui_email = input("Enter email          : ")
ui_product = input("Enter product name : ")
ui_quantity = input("Enter quantity    : ")


for i in products:
    product = i.split(',')

    if ui_product.lower() == product[1].lower():
        
        if int(ui_quantity) <= int(product[3]):
            print("\n\n")
            print("-"*13 + "BILLING" + "-"*14)
            print("Customer Name     : ", ui_name)
            print("Purchased Product : ", ui_product)
            print("Quantity          : ", ui_quantity)
            print("Price             : ", product[2])
            print("-" * 40)
            total_amount = int(ui_quantity) * int(product[2])
            print("Bill amount       : ", str(total_amount)+'\-')
            print("-" * 40)
            
            fd = open("Sales.txt", "a")
            sales_details = ui_name+","+ui_phone+","+ui_email+","+ui_product+","+ui_quantity+","+str(total_amount)+","+time.ctime()+"\n"
            fd.write(sales_details)
            fd.close()
            
            product[3] = str(int(product[3]) - int(ui_quantity))
        else:
            print("Sorry! We are not having enough stock")
            print("We are having only", product[3], "Quantity")
            print("Would you like to purchase ?")
            
            ch = input("Press Y/N")
            
            if ch == 'Y' or ch == 'y':
                print("-" * 40)
                print("Bill Details\n\n")
                print("-" * 40)
                print("Customer Name     : ", ui_name)
                print("Purchased Product : ", ui_product)
                print("Quantity          : ", product[3])
                print("Price             : ", product[2])
                print("-" * 40)
                total_amount = int(product[3]) * int(product[2])
                print("Bill amount       : ", str(total_amount)+'\-')
                print("-" * 40 + '\n\n')

                fd = open("Sales.txt", "a")
                sales_details = ui_name+","+ui_phone+","+ui_email+","+ui_product+","+ui_quantity+","+str(total_amount)+","+time.ctime()+"\n"
                fd.write(sales_details)
                fd.close()

                product[3] = '0'
        
    updated_products.append(product)
            
print("Thank You!!! Visit Again!!!")

lst = []
for i in updated_products:
    k = ','.join(i) + '\n'
    lst.append(k)
    
lst[-1] = lst[-1][:-1]

fd = open("Inventory.txt", "w")
for i in lst:
    fd.write(i)
fd.close()

print("Inventory Updated")

['1,munch,5,92', '2,dairymilk,10,150', '3,pastry,20,198', '4,milkybar,5,150', '5,icecream,30,200']



Enter user name       : ammu
Enter Phone number   : 345678
Enter email          : sdfghj
Enter product name : munch
Enter quantity    : 1



-------------BILLING--------------
Customer Name     :  ammu
Purchased Product :  munch
Quantity          :  1
Price             :  5
----------------------------------------
Bill amount       :  5\-
----------------------------------------
Thank You!!! Visit Again!!!
Inventory Updated


In [2]:
import time
import logging
import os
import shutil

# Configure logging
logging.basicConfig(filename='inventory.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')


def read_inventory(filename="Inventory.txt"):
    """Read inventory from the file and return as a list of products."""
    try:
        with open(filename, "r") as fd:
            txt = fd.read()
        products = [line.strip().split(',') for line in txt.split('\n') if line.strip()]
        logging.info("Inventory read successfully.")
        return products
    except FileNotFoundError:
        logging.error(f"{filename} not found. Creating a new inventory file.")
        open(filename, "w").close()  # Create the file if it doesn't exist
        return []
    except Exception as e:
        logging.error(f"Error reading {filename}: {e}")
        return []


def write_inventory(products, filename="Inventory.txt"):
    """Write the list of products back to the inventory file."""
    try:
        with open(filename, "w") as fd:
            for product in products:
                fd.write(','.join(product) + '\n')
        logging.info("Inventory updated successfully.")
    except Exception as e:
        logging.error(f"Error writing to {filename}: {e}")


def read_sales(filename="Sales.txt"):
    """Read sales records from the file."""
    try:
        with open(filename, "r") as fd:
            sales = [line.strip().split(',') for line in fd.readlines() if line.strip()]
        logging.info("Sales records read successfully.")
        return sales
    except FileNotFoundError:
        logging.error(f"{filename} not found. Creating a new sales file.")
        open(filename, "w").close()
        return []
    except Exception as e:
        logging.error(f"Error reading {filename}: {e}")
        return []


def write_sale(sale, filename="Sales.txt"):
    """Append a sale record to the sales file."""
    try:
        with open(filename, "a") as fd:
            fd.write(','.join(sale) + '\n')
        logging.info(f"Sale recorded: {sale}")
    except Exception as e:
        logging.error(f"Error writing to {filename}: {e}")


def authenticate(users):
    """Authenticate the user based on predefined users."""
    username = input("Username: ")
    password = input("Password: ")
    user = users.get(username)
    if user and user['password'] == password:
        print(f"Welcome {username}! Role: {user['role']}")
        logging.info(f"User '{username}' authenticated successfully as '{user['role']}'.")
        return user['role']
    else:
        print("Authentication failed!")
        logging.warning(f"Failed login attempt for username: {username}.")
        return None


def view_inventory(products):
    """Display the current inventory in a formatted table."""
    if not products:
        print("Inventory is empty.")
        return
    print("-" * 60)
    print(f"{'ID':<10}{'Name':<20}{'Price':<10}{'Quantity':<10}")
    print("-" * 60)
    for p in products:
        print(f"{p[0]:<10}{p[1]:<20}{p[2]:<10}{p[3]:<10}")
    print("-" * 60)


def add_product(products):
    """Add a new product to the inventory."""
    print("\n--- Add New Product ---")
    product_id = input("Enter product ID: ").strip()
    # Check if product ID already exists
    if any(p[0] == product_id for p in products):
        print("Product ID already exists. Please try again.")
        logging.warning(f"Attempt to add product with existing ID: {product_id}.")
        return products

    product_name = input("Enter product name: ").strip()
    while True:
        try:
            product_price = int(input("Enter product price: ").strip())
            if product_price < 0:
                raise ValueError
            break
        except ValueError:
            print("Invalid price. Please enter a positive integer.")
    while True:
        try:
            product_quantity = int(input("Enter product quantity: ").strip())
            if product_quantity < 0:
                raise ValueError
            break
        except ValueError:
            print("Invalid quantity. Please enter a positive integer.")

    products.append([product_id, product_name, str(product_price), str(product_quantity)])
    print("Product added successfully!")
    logging.info(f"Added product: {product_id}, {product_name}, Price: {product_price}, Quantity: {product_quantity}")
    return products


def remove_product(products):
    """Remove a product from the inventory."""
    print("\n--- Remove Product ---")
    product_name = input("Enter the product name to remove: ").strip().lower()
    initial_length = len(products)
    products = [p for p in products if p[1].lower() != product_name]
    if len(products) < initial_length:
        print("Product removed successfully!")
        logging.info(f"Removed product: {product_name}")
    else:
        print("Product not found.")
        logging.warning(f"Attempted to remove non-existent product: {product_name}")
    return products


def update_product(products):
    """Update details of an existing product."""
    print("\n--- Update Product ---")
    product_name = input("Enter the product name to update: ").strip().lower()
    for product in products:
        if product_name == product[1].lower():
            print("1. Update Price")
            print("2. Update Quantity")
            choice = input("Choose an option (1/2): ").strip()
            if choice == '1':
                while True:
                    try:
                        new_price = int(input("Enter new price: ").strip())
                        if new_price < 0:
                            raise ValueError
                        product[2] = str(new_price)
                        print("Price updated successfully!")
                        logging.info(f"Updated price for product '{product_name}' to {new_price}.")
                        break
                    except ValueError:
                        print("Invalid price. Please enter a positive integer.")
            elif choice == '2':
                while True:
                    try:
                        new_quantity = int(input("Enter new quantity: ").strip())
                        if new_quantity < 0:
                            raise ValueError
                        product[3] = str(new_quantity)
                        print("Quantity updated successfully!")
                        logging.info(f"Updated quantity for product '{product_name}' to {new_quantity}.")
                        break
                    except ValueError:
                        print("Invalid quantity. Please enter a positive integer.")
            else:
                print("Invalid choice.")
            break
    else:
        print("Product not found.")
        logging.warning(f"Attempted to update non-existent product: {product_name}")
    return products


def generate_sales_report(sales):
    """Generate and display a sales report."""
    if not sales:
        print("No sales records available.")
        return
    total_sales = sum(int(sale[5]) for sale in sales)
    product_sales = {}
    for sale in sales:
        product = sale[3]
        quantity = int(sale[4])
        product_sales[product] = product_sales.get(product, 0) + quantity

    best_selling = max(product_sales, key=product_sales.get) if product_sales else None

    print("\n--- Sales Report ---")
    print(f"Total Sales Amount: {total_sales} \-")
    if best_selling:
        print(f"Best Selling Product: {best_selling} ({product_sales[best_selling]} units)")
    print("--------------------")
    logging.info("Generated sales report.")


def backup_inventory(src="Inventory.txt", dest="Inventory_backup.txt"):
    """Backup the inventory file."""
    try:
        shutil.copy(src, dest)
        print("Inventory backed up successfully.")
        logging.info("Inventory backed up successfully.")
    except Exception as e:
        print("Failed to backup inventory.")
        logging.error(f"Failed to backup inventory: {e}")


def backup_sales(src="Sales.txt", dest="Sales_backup.txt"):
    """Backup the sales file."""
    try:
        shutil.copy(src, dest)
        print("Sales records backed up successfully.")
        logging.info("Sales records backed up successfully.")
    except Exception as e:
        print("Failed to backup sales records.")
        logging.error(f"Failed to backup sales records: {e}")


def sell_product(products, sales):
    """Process the sale of a product."""
    print("\n--- Sell Product ---")
    ui_name = input("Enter customer name     : ").strip()
    ui_phone = input("Enter phone number      : ").strip()
    ui_email = input("Enter email             : ").strip()
    ui_product = input("Enter product name     : ").strip()
    
    # Input validation for quantity
    while True:
        try:
            ui_quantity = int(input("Enter quantity          : ").strip())
            if ui_quantity <= 0:
                raise ValueError
            break
        except ValueError:
            print("Invalid quantity. Please enter a positive integer.")
    
    for product in products:
        if ui_product.lower() == product[1].lower():
            available_quantity = int(product[3])
            price = int(product[2])
            if ui_quantity <= available_quantity:
                total_amount = ui_quantity * price
                print("\n\n" + "-" * 13 + " BILLING " + "-" * 14)
                print(f"Customer Name     : {ui_name}")
                print(f"Purchased Product : {ui_product}")
                print(f"Quantity          : {ui_quantity}")
                print(f"Price per unit    : {price}")
                print("-" * 40)
                print(f"Total Amount      : {total_amount} \-")
                print("-" * 40 + "\n\n")

                # Record the sale
                sales_details = [
                    ui_name,
                    ui_phone,
                    ui_email,
                    ui_product,
                    str(ui_quantity),
                    str(total_amount),
                    time.ctime()
                ]
                write_sale(sales_details)

                # Update inventory
                product[3] = str(available_quantity - ui_quantity)
                print("Sale recorded successfully!")
                logging.info(f"Processed sale: {sales_details}")
            else:
                print("Sorry! We do not have enough stock.")
                print(f"We are having only {available_quantity} units.")
                choice = input("Would you like to purchase the available quantity? (Y/N): ").strip().lower()
                if choice == 'y':
                    total_amount = available_quantity * price
                    print("\n" + "-" * 40)
                    print("Bill Details\n")
                    print("-" * 40)
                    print(f"Customer Name     : {ui_name}")
                    print(f"Purchased Product : {ui_product}")
                    print(f"Quantity          : {available_quantity}")
                    print(f"Price per unit    : {price}")
                    print("-" * 40)
                    print(f"Total Amount      : {total_amount} \-")
                    print("-" * 40 + "\n\n")

                    # Record the partial sale
                    sales_details = [
                        ui_name,
                        ui_phone,
                        ui_email,
                        ui_product,
                        str(available_quantity),
                        str(total_amount),
                        time.ctime()
                    ]
                    write_sale(sales_details)

                    # Update inventory to zero
                    product[3] = '0'
                    print("Sale recorded successfully with available quantity!")
                    logging.info(f"Processed partial sale: {sales_details}")
            break
    else:
        print("Product not found in inventory.")
        logging.warning(f"Attempted to sell non-existent product: {ui_product}")
    
    return products, sales


def main_menu(role):
    """Display the main menu based on user role."""
    print("\n--- Inventory Management System ---")
    print("1. View Inventory")
    print("2. Sell Product")
    if role == 'admin':
        print("3. Add New Product")
        print("4. Remove Product")
        print("5. Update Product Details")
        print("6. Generate Sales Report")
        print("7. Backup Inventory")
        print("8. Backup Sales Records")
    print("0. Exit")


def main():
    """Main function to run the inventory management system."""
    # Define users with roles
    users = {
        'admin': {'password': 'adminpass', 'role': 'admin'},
        'sales': {'password': 'salespass', 'role': 'sales'}
    }

    # Authenticate user
    role = authenticate(users)
    if not role:
        return

    # Load inventory and sales data
    products = read_inventory()
    sales = read_sales()

    while True:
        main_menu(role)
        choice = input("Enter your choice: ").strip()

        if choice == '1':
            view_inventory(products)
        elif choice == '2':
            products, sales = sell_product(products, sales)
            write_inventory(products)
        elif choice == '3' and role == 'admin':
            products = add_product(products)
            write_inventory(products)
        elif choice == '4' and role == 'admin':
            products = remove_product(products)
            write_inventory(products)
        elif choice == '5' and role == 'admin':
            products = update_product(products)
            write_inventory(products)
        elif choice == '6' and role == 'admin':
            generate_sales_report(sales)
        elif choice == '7' and role == 'admin':
            backup_inventory()
        elif choice == '8' and role == 'admin':
            backup_sales()
        elif choice == '0':
            print("Exiting the system. Goodbye!")
            logging.info("User exited the system.")
            break
        else:
            print("Invalid choice or insufficient permissions. Please try again.")


if __name__ == "__main__":
    main()


Username: admin
Password: adminpass
Welcome admin! Role: admin

--- Inventory Management System ---
1. View Inventory
2. Sell Product
3. Add New Product
4. Remove Product
5. Update Product Details
6. Generate Sales Report
7. Backup Inventory
8. Backup Sales Records
0. Exit
Enter your choice: 2

--- Sell Product ---
Enter customer name     : amrutha
Enter phone number      : 234567
Enter email             : amrutha@gmail.com
Enter product name     : munch
Enter quantity          : 2


------------- BILLING --------------
Customer Name     : amrutha
Purchased Product : munch
Quantity          : 2
Price per unit    : 5
----------------------------------------
Total Amount      : 10 \-
----------------------------------------


Sale recorded successfully!

--- Inventory Management System ---
1. View Inventory
2. Sell Product
3. Add New Product
4. Remove Product
5. Update Product Details
6. Generate Sales Report
7. Backup Inventory
8. Backup Sales Records
0. Exit
Enter your choice: 0
Exitin